In [85]:
import pandas as pd
import numpy as np
import mysql.connector
import psycopg2
import json

In [86]:
##Sompo Prod
cnx = mysql.connector.connect(
    host= 'sompo-prod.ctcf2ufaplqs.ap-south-1.rds.amazonaws.com',
    port = 3306,
    database= 'cgi_pipeline_processor',
    user= 'read_user',
    password= '6QhOymR8euiz9zvamTQj')
client_id = '179' 

In [87]:
## Enter Date Range
start_date = "' 2024-06-20 '" #Chnage Start Date
end_date = " '2024-07-08'" #Chnage End Date

In [88]:
##Reprocess claims count
# For Date range

q1 = pd.read_sql('''select uid, count(uid) from claims where 
date(created_at) between ''' + start_date + ''' and ''' + end_date + ''' and 
client_id = ''' + client_id + ''' group by uid having count(uid) >1''',
con=cnx);
q1

C:\Users\surbhi.diwan\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,uid,count(uid)


In [5]:
####Removing extra claim_ids

q2 = pd.read_sql('''SELECT
    id, uid, date(created_at),is_final_version from claims 
    WHERE date(created_at) = ''' + date + ''' and client_id = ''' + client_id + ''' 
and uid in 
(
    SELECT uid from claims 
    WHERE date(created_at) = ''' + date + ''' and client_id = ''' + client_id + ''' 
GROUP BY uid 
HAVING count(uid) >1) 
ORDER BY uid''',
con=cnx);
q2

C:\Users\surbhi.diwan\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,uid,date(created_at),is_final_version


In [19]:
q2.shape

(2, 4)

In [20]:
def return_recent_claims(q2):
    final = pd.DataFrame()
    temp =q2.groupby(['uid'])[['is_final_version']].sum().reset_index()
#     print(temp.shape)
#     print()
    temp2 = temp[temp['is_final_version']==1].merge(q2).drop_duplicates()
#     print(temp2.shape)
    final = pd.concat([final,temp2])
#     print(final.shape)
    temp =q2[q2['uid'].isin(temp[temp['is_final_version']!=1].uid.unique())]
    final=pd.concat([final,temp[temp['id'].isin(temp.groupby(['uid'])['id'].max().to_list())]])
    return final
final =return_recent_claims(q2)

In [21]:
final.shape

(1, 4)

In [22]:
idstoskip = list(q2[~q2['id'].isin(final.id)].id.unique())

In [23]:
idstoskip

[105886]

In [24]:
len(idstoskip)

1

# RR Decisions

In [40]:
# Sompo

cp1 = pd.read_sql('''select date(created_at) as date, id as claim_id, uid, claim_number, make, 
model, year, body_type, rr_prediction from claims where status in (2,3) and client_id = ''' + client_id + ''' and 
date(created_at) between ''' + start_date + ''' and ''' + end_date + ''' and 
rr_prediction is not null;''',
 con=cnx);
cp1


## Sompo - if reprocess claims are present

# cp1 = pd.read_sql('''select date(created_at) as date, id as claim_id, uid, claim_number, make, model, year, body_type, 
# client_id, rr_prediction from claims where client_id = ''' + client_id + '''
# and date(created_at) between ''' + start_date + ''' and ''' + end_date + ''' and id not in ''' + str(tuple(idstoskip)) + ''' 
# and rr_prediction is not null '''
#  con=cnx);
# cp1



C:\Users\surbhi.diwan\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,date,claim_id,uid,claim_number,make,model,year,body_type,rr_prediction
0,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,"[{""damages"":{},""decision"":""part_damage_not_ide..."
1,2024-06-25,104,CKF3BBVL,CL24149045,MARUTI SUZUKI,BALENO,2017,hatchback,"[{""damages"":{},""decision"":""part_damage_not_ide..."
2,2024-06-25,105,CSRGG6GG,CL24122681,MARUTI SUZUKI,VITARA BREZZA,2020,suv,"[{""damages"":{""ch"":0,""dent"":4,""dent_damage_thre..."
3,2024-06-25,106,CSRHS6QW,CL24135093,MARUTI SUZUKI,BALENO,2021,hatchback,"[{""damages"":{},""decision"":""part_damage_not_ide..."
4,2024-06-25,107,CZ139YLJ,CL24115902,MARUTI SUZUKI,CELERIO,2018,hatchback,"[{""damages"":{},""decision"":""part_not_identified..."
...,...,...,...,...,...,...,...,...,...
549,2024-07-08,660,C3KLHAX1,CL24171483,MARUTI SUZUKI,S-CROSS,2018,suv,"[{""damages"":{},""decision"":""part_damage_not_ide..."
550,2024-07-08,661,C48HHG5H,CL24172788,MARUTI SUZUKI,S-CROSS,2018,suv,"[{""damages"":{""ch"":0,""dent"":1,""dent_damage_thre..."
551,2024-07-08,662,CDS8MKLN,CL24173047,MARUTI SUZUKI,CIAZ,2015,sedan,"[{""damages"":{},""decision"":""part_damage_not_ide..."
552,2024-07-08,663,C311QXB1,CL24173261,MARUTI SUZUKI,BALENO,2016,hatchback,"[{""damages"":{},""decision"":""part_not_identified..."


In [7]:
cp1 = cp1.head(330)   ### Add number of claims to be extracted for curation - number of days * 30
cp1

,date,claim_id,uid,claim_number,make,model,year,body_type,rr_prediction
0,2024-06-28,156,CIRKIVB3,CL24158801,MARUTI SUZUKI,VITARA BREZZA,2017,suv,"[{""damages"":{""ch"":0,""dent"":0,""dent_damage_thre..."
1,2024-06-28,157,C59Q5MWG,CL24158110,MARUTI SUZUKI,SWIFT,2018,hatchback,"[{""damages"":{},""decision"":""part_damage_not_ide..."
2,2024-06-28,158,C6Z3LZXZ,CL24158080,MARUTI SUZUKI,WAGON R,2021,hatchback,"[{""damages"":{},""decision"":""part_damage_not_ide..."
3,2024-06-28,159,CWG6WI5K,CL24158987,MARUTI SUZUKI,ALTO 800,2017,hatchback,"[{""damages"":{},""decision"":""part_damage_not_ide..."
4,2024-06-28,160,CWMOX23J,CL24159028,MARUTI SUZUKI,SWIFT DZIRE,2016,sedan,"[{""damages"":{},""decision"":""part_damage_not_ide..."
...,...,...,...,...,...,...,...,...,...
325,2024-07-06,485,CHFZKUWZ,CL24170364,MARUTI SUZUKI,VITARA BREZZA,2018,suv,"[{""damages"":{""ch"":0,""dent"":1,""dent_damage_thre..."
326,2024-07-06,486,CQSQGJ12,CL24170387,MARUTI SUZUKI,SWIFT,2011,hatchback,"[{""damages"":{},""decision"":""part_not_identified..."
327,2024-07-06,487,CLXYNC9B,CL24170180,FORD,ASPIRE,2018,sedan,"[{""damages"":{},""decision"":""part_damage_not_ide..."
328,2024-07-06,488,CGWIK9AB,CL24170546,MARUTI SUZUKI,VITARA BREZZA,2016,suv,"[{""damages"":{},""decision"":""part_damage_not_ide..."


In [41]:
from tqdm import tqdm

In [42]:
date=[]
claim_id=[]
uid=[]
claim_number=[]
make = []
model = []
year = []
# client_id = []
#vin= []
part_name=[]
rr_decision=[]
scratch=[]
dent=[]
tear=[]
body_type=[]

for i in tqdm(range(len(cp1))):
    for j in json.loads(cp1.iloc[i]['rr_prediction']):
        body_type.append(cp1.iloc[i]['body_type'])
        date.append(cp1.iloc[i]['date'])
        claim_id.append(cp1.iloc[i]['claim_id'])
        uid.append(cp1.iloc[i]['uid'])
        claim_number.append(cp1.iloc[i]['claim_number'])
        make.append(cp1.iloc[i]['make'])
        model.append(cp1.iloc[i]['model'])
        year.append(cp1.iloc[i]['year'])
#         client_id.append(cp1.iloc[i]['client_id'])
        #vin.append(cp1.iloc[i]['vin'])
        part_name.append(j['part_name'])
        rr_decision.append(j['decision'])
        if len(j['damages'])==0:
            scratch.append(0)
            dent.append(0)
            tear.append(0)
        else:
            scratch.append(j['damages']['scratch'])
            dent.append(j['damages']['dent'])
            tear.append(j['damages']['tear'])
                              
claim_decision1=pd.DataFrame({'date':date,'claim_id':claim_id,'uid': uid, 'claim_number': claim_number,'make': make, 'model':model,'year': year, 'body_type': body_type, 'part_name':part_name,
                            'rr_decision':rr_decision,'scratch_count':scratch,'dent_count':dent,'tear_count':tear })
        
display(claim_decision1)            

100%|██████████| 554/554 [00:27<00:00, 19.95it/s]


,date,claim_id,uid,claim_number,make,model,year,body_type,part_name,rr_decision,scratch_count,dent_count,tear_count
0,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,bonnet,part_damage_not_identified,0,0,0
1,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,bumper,part_damage_not_identified,0,0,0
2,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,door_front_left,part_damage_not_identified,0,0,0
3,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,door_front_right,part_damage_not_identified,0,0,0
4,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,door_rear_left,part_damage_not_identified,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28803,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,hatchback,window_front_right,part_damage_not_identified,0,0,0
28804,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,hatchback,window_rear_left,part_damage_not_identified,0,0,0
28805,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,hatchback,window_rear_right,part_damage_not_identified,0,0,0
28806,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,hatchback,windshield,replace,0,0,0


In [43]:
claim_decision1.nunique()

date              12
claim_id         554
uid              554
claim_number     554
make              10
model             65
year              16
body_type          5
part_name         52
rr_decision        4
scratch_count     38
dent_count         7
tear_count        14
dtype: int64

In [44]:
claim_decision = claim_decision1.drop_duplicates(subset=['claim_number','part_name'], keep = 'last')
claim_decision

,date,claim_id,uid,claim_number,make,model,year,body_type,part_name,rr_decision,scratch_count,dent_count,tear_count
0,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,bonnet,part_damage_not_identified,0,0,0
1,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,bumper,part_damage_not_identified,0,0,0
2,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,door_front_left,part_damage_not_identified,0,0,0
3,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,door_front_right,part_damage_not_identified,0,0,0
4,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,door_rear_left,part_damage_not_identified,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28803,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,hatchback,window_front_right,part_damage_not_identified,0,0,0
28804,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,hatchback,window_rear_left,part_damage_not_identified,0,0,0
28805,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,hatchback,window_rear_right,part_damage_not_identified,0,0,0
28806,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,hatchback,windshield,replace,0,0,0


In [45]:
claim_decision.nunique()

date              12
claim_id         554
uid              554
claim_number     554
make              10
model             65
year              16
body_type          5
part_name         52
rr_decision        4
scratch_count     38
dent_count         7
tear_count        14
dtype: int64

In [46]:
claim_decision.to_excel(r"E:\CG\POC\Sompo\DailyCuration/SompoProd_rrdecisions_allclaims.xlsx", index = False)

# UPD Decisions

In [89]:
# claim= pd.read_sql('''
# select distinct id,client_id,date(created_at) as date,uid,id as claim_id,claim_number,make,model,year,body_type,
# part_pricing,is_final_version from claims where date(created_at) between ''' + start_date + ''' and 
# ''' + end_date + ''' and status in (2,3) and rr_prediction is not null and id in ''' + str(tuple(claim_decision['claim_id'])) + 
#                    '''
# '''
#  , con=cnx);


# cnx.close()
# claim

claim= pd.read_sql('''
select distinct id,client_id,date(created_at) as date,uid,id as claim_id,claim_number,make,model,year,body_type,
part_pricing,is_final_version from claims where client_id = ''' + client_id + ''' and 
date(created_at) between ''' + start_date + ''' and ''' + end_date + ''' and status in (2,3) and rr_prediction is not null '''
 , con=cnx);


cnx.close()
claim

C:\Users\surbhi.diwan\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,client_id,date,uid,claim_id,claim_number,make,model,year,body_type,part_pricing,is_final_version
0,103,179,2024-06-25,CZ6ZVDBP,103,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,suv,"{""summary"":{""total_net_price"":0,""total_part_pr...",0
1,104,179,2024-06-25,CKF3BBVL,104,CL24149045,MARUTI SUZUKI,BALENO,2017,hatchback,"{""summary"":{""total_net_price"":0,""total_part_pr...",0
2,105,179,2024-06-25,CSRGG6GG,105,CL24122681,MARUTI SUZUKI,VITARA BREZZA,2020,suv,"{""summary"":{""total_net_price"":7975,""total_part...",0
3,106,179,2024-06-25,CSRHS6QW,106,CL24135093,MARUTI SUZUKI,BALENO,2021,hatchback,"{""summary"":{""total_net_price"":8415,""total_part...",0
4,107,179,2024-06-25,CZ139YLJ,107,CL24115902,MARUTI SUZUKI,CELERIO,2018,hatchback,"{""summary"":{""total_net_price"":6090,""total_part...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
549,660,179,2024-07-08,C3KLHAX1,660,CL24171483,MARUTI SUZUKI,S-CROSS,2018,suv,"{""summary"":{""total_net_price"":0,""total_part_pr...",0
550,661,179,2024-07-08,C48HHG5H,661,CL24172788,MARUTI SUZUKI,S-CROSS,2018,suv,"{""summary"":{""total_net_price"":9890,""total_part...",0
551,662,179,2024-07-08,CDS8MKLN,662,CL24173047,MARUTI SUZUKI,CIAZ,2015,sedan,"{""summary"":{""total_net_price"":0,""total_part_pr...",0
552,663,179,2024-07-08,C311QXB1,663,CL24173261,MARUTI SUZUKI,BALENO,2016,hatchback,"{""summary"":{""total_net_price"":9040,""total_part...",0


In [90]:
claim_id=[]
claim_number=[]
client_id = []
date = []
uid = []
make = []
model = []
year = []
body_type = []
part_name=[]
included=[]
decision = []
uid=[]

for i in tqdm(range(len(claim))):
    #print(i)
    #print(type(claims_set12.iloc[i]['rr_prediction']))
    parts = json.loads(claim.iloc[i]['part_pricing'])
    parts = parts['parts']
    for j in parts:
        claim_id.append(claim.iloc[i]['claim_id'])
        claim_number.append(claim.iloc[i]['claim_number'])
        uid.append(claim.iloc[i]['uid'])
        date.append(claim.iloc[i]['date'])
        make.append(claim.iloc[i]['make'])
        model.append(claim.iloc[i]['model'])
        year.append(claim.iloc[i]['year'])
        client_id.append(claim.iloc[i]['client_id'])
        body_type.append(claim.iloc[i]['body_type'])
        part_name.append(j['part_std_name'])
        decision.append(j['decision'])
        included.append(j['is_included'])
#         uid.append(claim.iloc[i]['uid'])
claim_upd_decision=pd.DataFrame({'date':date, 'claim_id': claim_id, 'uid':uid,'claim_number':claim_number,'make':make,
                                 'model':model,'year':year, 'client_id':client_id,'body_type':body_type,'part_name':part_name,
                                 'included':included, 'decision':decision})
        
claim_upd_decision

100%|██████████| 554/554 [00:29<00:00, 18.57it/s]


,date,claim_id,uid,claim_number,make,model,year,client_id,body_type,part_name,included,decision
0,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,bonnet,False,part_damage_not_identified
1,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,bumper,False,part_damage_not_identified
2,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,door_front_left,False,part_damage_not_identified
3,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,door_front_right,False,part_damage_not_identified
4,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,door_rear_left,False,part_damage_not_identified
...,...,...,...,...,...,...,...,...,...,...,...,...
28803,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,179,hatchback,airbag,False,part_not_identified
28804,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,179,hatchback,door_rear_left,True,repair
28805,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,179,hatchback,fender_rear_left,True,repair
28806,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,179,hatchback,rear_bumper,True,replace


In [91]:
claim_upd_decision.nunique()

date             12
claim_id        554
uid             554
claim_number    554
make             10
model            65
year             16
client_id         1
body_type         5
part_name        52
included          2
decision          4
dtype: int64

In [92]:
parts_list = ['bonnet', 'bumper', 'bumper_lower', 'bumper_upper', 'door_front_left', 'door_front_right', 'door_rear_left', 
              'door_rear_right', 'fender_front_left', 'fender_front_right', 'fender_rear_left', 'fender_rear_right', 
              'headlight_left', 'headlight_right', 'rear_bumper', 'rear_windshield', 'running_board_left', 'running_board_right', 'side_panel_cargo_left', 'side_panel_cargo_right',
              'tail_light_left', 'tail_light_right', 'trunk_cover', 'windshield'               
          ]

In [93]:
claim_upd_decision = claim_upd_decision[claim_upd_decision['part_name'].isin(parts_list)]
claim_upd_decision

,date,claim_id,uid,claim_number,make,model,year,client_id,body_type,part_name,included,decision
0,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,bonnet,False,part_damage_not_identified
1,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,bumper,False,part_damage_not_identified
2,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,door_front_left,False,part_damage_not_identified
3,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,door_front_right,False,part_damage_not_identified
4,2024-06-25,103,CZ6ZVDBP,CL24142722,MARUTI SUZUKI,GRAND VITARA,2023,179,suv,door_rear_left,False,part_damage_not_identified
...,...,...,...,...,...,...,...,...,...,...,...,...
28794,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,179,hatchback,trunk_cover,False,part_damage_not_identified
28804,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,179,hatchback,door_rear_left,True,repair
28805,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,179,hatchback,fender_rear_left,True,repair
28806,2024-07-08,664,C7YBSMJJ,CL24173039,MARUTI SUZUKI,CELERIO,2018,179,hatchback,rear_bumper,True,replace


In [94]:
claim_upd_decision.nunique()

date             12
claim_id        554
uid             554
claim_number    554
make             10
model            65
year             16
client_id         1
body_type         5
part_name        20
included          2
decision          4
dtype: int64

In [97]:
claim_upd_decision.to_excel(r"E:\CG\POC\Sompo\DailyCuration/UPD_decisions_Sompo_allUPDdecisions.xlsx", index = False)